## 1-B

In [99]:
from math import ceil
from copy import deepcopy
import pandas as pd
import numpy as np

# Read data from csv file
data = pd.read_csv('fods_1.csv')

# Convert data to numpy array
data = data.to_numpy()

# Shuffle and split data
np.random.shuffle(data)
N = data.shape[0]
N_train = ceil(N * 0.8)
N_test = N - N_train
X_train = data[:N_train,0:2]
y_train = data[:N_train,2]
X_test = data[N_train:,0:2]
y_test = data[N_train:,2]

print(f"X_t : {np.average(X_train, axis = 0)} y_t : {np.average(y_train)}")

X_t : [2.32570938 2.4809016 ] y_t : 4.620254004576659


### Cost
We add a dummy input variable $x_0^{(i)} = 1$ to all $N$ examples in order to handle bias.
$$ 
f_{w}(x^{(i)}) = w_0 x_0^{(i)} + w_1 x_1^{(i)} + \cdots + w_d x_d^{(i)} \\
J_{w} = \frac{1}{2N} \sum_{i=1}^{N} (f_{w}(x^{(i)}) - y^{(i)})^2 
$$

In [104]:
def compute_cost(X, y, w):
  """
  compute cost
  Args:
    X : (ndarray): Shape (N,d) matrix of examples with multiple features
    w : (ndarray): Shape (d,1)   parameters for prediction   
  Returns
    cost: (scalar)             cost
  """
  N = X.shape[0]
  f_w = X @ w
  # cost = (1 / (2 * N)) * np.sum((f_w - y) ** 2)
  cost = np.sum((f_w - y) ** 2 / (2 * N))
  return cost

### Gradient

$$ 
\frac{\partial J(w,b)}{\partial w_j}  = \frac{1}{N} \sum\limits_{i = 1}^{N} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)}_j \\

$$

In [101]:
def compute_gradient(X, y, w):
  """
  Computes the gradient for linear regression 

  Args:
    X : (array_like Shape (N,d)) variable such as house size 
    y : (array_like Shape (N,1)) actual value 
    w : (array_like Shape (d,1)) Values of parameters of the model      
  Returns
    dj_dw: (array_like Shape (d,1)) The gradient of the cost w.r.t. the parameters w. 
                                
  """
  N, d = X.shape
  f_w = X @ w
  # dj_dw = (1 / N) * (X.T @ (f_w - y))
  dj_dw = ((X.T / N) @ (f_w - y))
  return dj_dw

In [122]:
def grad_descent(X, y, w_in, cost_fn, grad_fn, alpha, num_iters):
  """
  Performs batch gradient descent to learn theta. Updates theta by taking 
  num_iters gradient steps with learning rate alpha

  Args:
    X : (array_like Shape (N,d)    matrix of examples 
    y : (array_like Shape (N,))    target value of each example
    w_in : (array_like Shape (d,)) Initial values of parameters of the model
    cost_fn: function to compute cost
    grad_fn: function to compute the gradient
    alpha : (float) Learning rate
    num_iters : (int) number of iterations to run gradient descent
  Returns
    w : (array_like Shape (d,)) Updated values of parameters of the model after
        running gradient descent
  """
  N, d = X.shape
    
  # An array to store values at each iteration primarily for graphing later
  hist = {}
  hist["cost"] = []; hist["params"] = []; hist["grads"] = []; hist["iter"] = [];
  
  w = deepcopy(w_in)  #avoid modifying global w within function
  save_interval = np.ceil(num_iters / 1000) # prevent resource exhaustion for long runs
  last = cost_fn(X, y, w)

  for i in range(num_iters):

      # Calculate the gradient and update the parameters
      dj_dw = grad_fn(X, y, w)   

      # Update Parameters using w, b, alpha and gradient
      w = w - alpha * dj_dw              
    
      # Save cost J,w,b at each save interval for graphing
      if i == 0 or i % save_interval == 0:   
          hist["cost"].append(cost_fn(X, y, w))
          hist["params"].append([w])
          hist["grads"].append([dj_dw])
          hist["iter"].append(i)

      # Print cost every at intervals 10 times or as many iterations if < 10
      if i % ceil(num_iters/10) == 0:
          cst = cost_fn(X, y, w)
          if cst > last * 10:
            print(f"**Diverging at alpha: {alpha}")
            return w, hist
          last = cst
          # print(f"Iteration {i:4d}: Cost {cst:8.2f}   ")
          # with np.printoptions(precision=3, suppress=True):
          #   print(f"w: {w}  dj/dw: {dj_dw}")
          
      
  return w, hist #return w,b and history for graphing

In [126]:
max_degree = 10
X_cur = np.ones(N_train).reshape(-1, 1)
alpha_count = 5
alpha_vector = np.zeros(alpha_count)
alpha_vector[0] = 1e-5
for i in range(1, alpha_count):
    alpha_vector[i] = alpha_vector[i-1] * 10
num_iters = 1000
cost_matrix = np.zeros((max_degree, alpha_count))
# print(X_train[:5])
for degree in range(max_degree):
    if degree > 0:
        for d1 in range(degree + 1):
            d2 = degree - d1
            X1new = (X_train[:,0] ** d1)
            X2new = (X_train[:,1] ** d2)
            X_cur = np.append(X_cur, (X1new * X2new).reshape(-1, 1), axis=1)
    if degree == 2:
        alpha_vector /= 100
    if degree > 3:
        alpha_vector /= 10
    if degree > 5:
        alpha_vector /= 10
    if degree > 7:
        alpha_vector /= 10
    d = X_cur.shape[1]
    print(f"Degree: {degree}, d: {d}")
    w_in = np.zeros(d)
    for a_i in range(alpha_count):
        # print(f"Alpha: {alpha_vector[a_i]}")
        w, hist = grad_descent(X_cur, y_train, w_in, compute_cost, compute_gradient, alpha_vector[a_i], num_iters)
        cost_matrix[degree][a_i] = compute_cost(X_cur, y_train, w)
        # print(f"Cost: {cost_matrix[degree][a_i] : .3f}")
        # with np.printoptions(precision=3, suppress=True):
        #     print(f"w: {w}")
    print(f"Alpha: {alpha_vector}")
    with np.printoptions(precision=3, suppress=True):
        print(f"Cost: {cost_matrix[degree]}")
    print("\n")

Degree: 0, d: 1
Alpha: [1.e-05 1.e-04 1.e-03 1.e-02 1.e-01]
Cost: [11.839 10.115  2.82   1.377  1.377]


Degree: 1, d: 3
Alpha: [1.e-05 1.e-04 1.e-03 1.e-02 1.e-01]
Cost: [9.307 1.826 1.074 0.932 0.9  ]


Degree: 2, d: 6
Alpha: [1.e-07 1.e-06 1.e-05 1.e-04 1.e-03]
Cost: [11.585  8.373  3.196  2.384  1.06 ]


Degree: 3, d: 10
**Diverging at alpha: 0.001
Alpha: [1.e-07 1.e-06 1.e-05 1.e-04 1.e-03]
Cost: [  7.715   5.545   3.978   1.483 235.194]


Degree: 4, d: 15
**Diverging at alpha: 1e-05
**Diverging at alpha: 0.0001
Alpha: [1.e-08 1.e-07 1.e-06 1.e-05 1.e-04]
Cost: [8.879e+00 7.477e+00 4.642e+00 1.058e+70 2.533e+03]


Degree: 5, d: 21
**Diverging at alpha: 1.0000000000000002e-06
**Diverging at alpha: 1e-05
Alpha: [1.e-09 1.e-08 1.e-07 1.e-06 1.e-05]
Cost: [    9.953     8.204     5.401   379.316 40755.164]


Degree: 6, d: 28
**Diverging at alpha: 1.0000000000000002e-08
**Diverging at alpha: 1.0000000000000002e-07
Alpha: [1.e-11 1.e-10 1.e-09 1.e-08 1.e-07]
Cost: [1.126e+001 1.033e+001